In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
from resnet import build_resunet
from metricsres import dice_loss, dice_coef, iou

In [2]:
""" Global Parameters """
H = 256
W = 256

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(path, split=0.2):
    images = sorted(glob(os.path.join(path, r"C:\Users\User\Desktop\cubaan2\data\images", "*.png")))
    masks = sorted(glob(os.path.join(path, r"C:\Users\User\Desktop\cubaan2\data\mask", "*.png")))

    split_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=4):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(10)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("filesResnet")

    """ Hyperparameters """
    batch_size = 12
    lr = 0.01
    num_epochs = 5
    model_path = os.path.join("filesResnet", "modelresunet1.h5")
    csv_path = os.path.join("filesResnet", "dataresunet1.csv")

    """ Dataset """
    dataset_path = r"C:\Users\User\Desktop\cubaan2\data"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)
    train_x, train_y = shuffling(train_x, train_y)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_dataset)
    valid_steps = len(valid_dataset)

    """ Model """
    model = build_resunet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train: 1376 - 1376
Valid: 458 - 458
Test: 458 - 458
Epoch 1/5
115/115 [==============================] - ETA: 0s - loss: 0.1992 - dice_coef: 0.8008 - iou: 0.6710 - recall: 0.9984 - precision: 0.6721 
Epoch 1: val_loss improved from inf to 0.20149, saving model to filesResnet\modelresunet1.h5
115/115 [==============================] - 3933s 34s/step - loss: 0.1992 - dice_coef: 0.8008 - iou: 0.6710 - recall: 0.9984 - precision: 0.6721 - val_loss: 0.2015 - val_dice_coef: 0.7993 - val_iou: 0.6709 - val_recall: 1.0000 - val_precision: 0.6699 - lr: 0.0100
Epoch 2/5
115/115 [==============================] - ETA: 0s - loss: 0.1984 - dice_coef: 0.8016 - iou: 0.6719 - recall: 1.0000 - precision: 0.6720 
Epoch 2: val_loss did not improve from 0.20149
115/115 [==============================] - 3768s 33s/step - loss: 0.1984 - dice_coef: 0.8016 - iou: 0.6719 - recall: 1.0000 - precision: 0.6720 - val_loss: 0.2015 - val_dice_coef: 0.7993 - val_iou: 0.6709 - val_recall: 1.0000 - val_precision: 0.6699